In [33]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import graphviz
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from datetime import datetime

pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

#### Read and pre-process data

In [34]:
# fetch data 

custom_parser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

flight_data = pd.read_csv('preprocssed_flight_weather.csv', parse_dates=['Date','time'], date_parser = custom_parser)
flight_data = flight_data.rename(columns={'Departure delay (cat)': 'Status', 'Origin AirportArr':'Origin Airport','Destination Airport_Dep':'Destination Airport'})


def str_to_time(x):
    return datetime.strptime(x, '%H:%M').time()

flight_data['Scheduled Arrival Time'] = flight_data['Scheduled Arrival Time'].astype(str).apply(lambda x: datetime.strptime(x, "%H:%M").time())
flight_data['Scheduled departure time'] = flight_data['Scheduled departure time'].astype(str).apply(lambda x: datetime.strptime(x, "%H:%M").time())

flight_data['Flight Number'] = flight_data['Flight Number'].astype(str)

In [35]:
#datetime.time values get represented only as object
#Extract hour and minute components from two columns

flight_data['Scheduled Arrival(Hrs)'] = flight_data['Scheduled Arrival Time'].apply(lambda x: x.hour)
flight_data['Scheduled Arrival(Mins)'] = flight_data['Scheduled Arrival Time'].apply(lambda x: x.minute)
flight_data['Scheduled Departure(Hrs)'] = flight_data['Scheduled departure time'].apply(lambda x: x.hour)
flight_data['Scheduled Departure(Mins)'] = flight_data['Scheduled departure time'].apply(lambda x: x.minute)

In [36]:
flight_data = flight_data.drop(columns=['Scheduled Arrival Time','Scheduled departure time','Origin Airport_Dep','Carrier Code_Arr','Scheduled Elapsed Time (Minutes)','Scheduled elapsed time (Minutes)','Scheduled departure hour','Destination Airport','time','Unnamed: 0','Departure delay (Minutes)','Arrival Delay (Minutes)'])
#flight_data['Status'] = flight_data['Status'].replace({'early': '1', 'on-time': '1','delayed': '2','severely delayed': '3'})

In [37]:
flight_data.head()

,Unnamed: 0.1,Date,Flight Number,Origin Airport,temperature_2m (°C),surface_pressure (hPa),rain (mm),snowfall (cm),weathercode (wmo code),windspeed_10m (km/h),windspeed_100m (km/h),Status,Arrival delay (cat),Scheduled Arrival(Hrs),Scheduled Arrival(Mins),Scheduled Departure(Hrs),Scheduled Departure(Mins)
0,0,1988-01-01,554,ORD,-16.60,"1,007.20",0.00,0.00,1,17.30,24.90,on-time,early,19,33,16,56
1,1,1988-01-01,700,ORD,-14.90,"1,007.20",0.00,0.00,1,19.90,27.90,delayed,on-time,22,34,19,57
2,2,1988-01-02,554,ORD,-11.90,"1,005.70",0.00,0.00,0,18.50,31.80,on-time,on-time,19,33,16,56
3,3,1988-01-02,656,ORD,-16.90,"1,006.30",0.00,0.00,0,14.10,29.50,on-time,on-time,16,16,13,44
4,4,1988-01-02,700,ORD,-6.30,"1,002.80",0.00,0.00,0,23.10,34.50,on-time,on-time,22,34,19,57


In [38]:
from sklearn.preprocessing import OneHotEncoder

def get_ohe(df, col):
    ohe = OneHotEncoder(drop=None, handle_unknown='error', sparse=False, dtype='int')
    ohe.fit(df[[col]])
    temp_df = pd.DataFrame(data=ohe.transform(df[[col]]), columns=ohe.get_feature_names_out())
    # If you have a newer version, replace with columns=ohe.get_feature_names_out()
    df.drop(columns=[col], axis=1, inplace=True)
    df = pd.concat([df.reset_index(drop=True), temp_df], axis=1)
    return df

#flight_data = get_ohe(flight_data, 'Carrier Code_Arr')
flight_data = get_ohe(flight_data, 'Origin Airport')
flight_data.head(5)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Unnamed: 0.1,Date,Flight Number,temperature_2m (°C),surface_pressure (hPa),rain (mm),snowfall (cm),weathercode (wmo code),windspeed_10m (km/h),windspeed_100m (km/h),Status,Arrival delay (cat),Scheduled Arrival(Hrs),Scheduled Arrival(Mins),Scheduled Departure(Hrs),Scheduled Departure(Mins),Origin Airport_DEN,Origin Airport_EWR,Origin Airport_IAD,Origin Airport_ORD
0,0,1988-01-01,554,-16.60,"1,007.20",0.00,0.00,1,17.30,24.90,on-time,early,19,33,16,56,0,0,0,1
1,1,1988-01-01,700,-14.90,"1,007.20",0.00,0.00,1,19.90,27.90,delayed,on-time,22,34,19,57,0,0,0,1
2,2,1988-01-02,554,-11.90,"1,005.70",0.00,0.00,0,18.50,31.80,on-time,on-time,19,33,16,56,0,0,0,1
3,3,1988-01-02,656,-16.90,"1,006.30",0.00,0.00,0,14.10,29.50,on-time,on-time,16,16,13,44,0,0,0,1
4,4,1988-01-02,700,-6.30,"1,002.80",0.00,0.00,0,23.10,34.50,on-time,on-time,22,34,19,57,0,0,0,1


In [42]:
flight_data['Date'] = pd.to_numeric(pd.to_datetime(flight_data['Date']))
order = ['Date','Flight Number','temperature_2m (°C)','surface_pressure (hPa)','rain (mm)','snowfall (cm)','weathercode (wmo code)','windspeed_10m (km/h)','windspeed_100m (km/h)','Scheduled Arrival(Hrs)','Scheduled Arrival(Mins)','Scheduled Departure(Hrs)','Scheduled Departure(Mins)','Origin Airport_DEN','Origin Airport_EWR','Origin Airport_IAD','Origin Airport_ORD','Status']
flight_data = flight_data.reindex(columns=order)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(flight_data.drop(columns = ['Status']), flight_data['Status'], test_size=0.25, random_state=35)
# X_train, X_test, y_train, y_test = train_test_split(subset_data.drop(columns = ['SalePrice', 'x0_Reg', 'x0_Detchd', 'x0_Other']), subset_data['SalePrice'], test_size=0.25, random_state=35)
# Pick a random_state as below and keep using the same number (example 35) to repeat the same test and train data
# X_train, X_test, y_train, y_test = train_test_split(subset_data['bmi'], subset_data['charges'], test_size=0.25, random_state=35)
X_train
X_test
y_train
y_test

,Date,Flight Number,temperature_2m (°C),surface_pressure (hPa),rain (mm),snowfall (cm),weathercode (wmo code),windspeed_10m (km/h),windspeed_100m (km/h),Scheduled Arrival(Hrs),Scheduled Arrival(Mins),Scheduled Departure(Hrs),Scheduled Departure(Mins),Origin Airport_DEN,Origin Airport_EWR,Origin Airport_IAD,Origin Airport_ORD
2525,702345600000000000,1620,-0.10,976.40,0.00,0.00,1,4.30,8.00,9,26,6,45,0,0,0,1
5335,979430400000000000,356,1.70,988.30,0.20,0.00,51,13.30,25.00,16,9,13,20,0,0,0,1
100,570240000000000000,700,-9.00,"1,004.50",0.00,0.00,0,15.30,20.70,22,27,19,50,0,0,0,1
1402,611107200000000000,1070,20.70,"1,004.20",0.00,0.00,1,7.90,11.90,22,48,21,40,0,0,1,0
5571,986169600000000000,1620,1.00,990.10,0.00,0.00,0,7.10,10.30,9,25,6,45,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321,766540800000000000,356,8.60,995.30,0.00,0.00,0,14.40,25.10,16,32,13,45,0,0,0,1
3007,758764800000000000,1498,-19.40,994.00,0.00,0.00,3,22.60,32.90,19,35,16,45,0,0,0,1
951,602208000000000000,906,15.00,972.80,0.00,0.00,0,28.80,43.00,22,29,19,50,0,0,0,1
1295,607910400000000000,906,6.20,988.80,0.00,0.00,1,3.60,4.30,22,33,20,0,0,0,0,1


,Date,Flight Number,temperature_2m (°C),surface_pressure (hPa),rain (mm),snowfall (cm),weathercode (wmo code),windspeed_10m (km/h),windspeed_100m (km/h),Scheduled Arrival(Hrs),Scheduled Arrival(Mins),Scheduled Departure(Hrs),Scheduled Departure(Mins),Origin Airport_DEN,Origin Airport_EWR,Origin Airport_IAD,Origin Airport_ORD
3379,789955200000000000,1620,0.80,983.50,0.00,0.00,3,5.20,13.40,10,30,7,49,0,0,0,1
3047,759628800000000000,1620,-8.00,997.30,0.00,0.00,3,23.80,35.40,9,26,6,44,0,0,0,1
5934,1018915200000000000,1498,20.80,988.10,0.00,0.00,0,20.30,37.50,9,42,7,5,0,0,0,1
6103,1581811200000000000,1643,0.90,995.60,0.00,0.00,0,1.50,2.20,21,19,18,30,0,0,0,1
1837,636163200000000000,750,-1.40,"1,010.70",0.00,0.00,0,6.40,8.50,22,43,20,5,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,596592000000000000,1070,17.90,996.90,0.00,0.00,3,10.90,21.80,22,33,21,25,0,0,1,0
3845,824947200000000000,752,1.70,993.20,0.00,0.00,3,12.70,17.70,20,28,17,44,0,0,0,1
6642,1663545600000000000,2360,28.70,"1,003.60",0.00,0.00,2,12.10,17.10,18,35,17,18,0,0,1,0
6720,1672704000000000000,604,-5.70,832.00,0.00,0.00,1,7.80,16.80,14,58,9,45,1,0,0,0


2525             on-time
5335             delayed
100     severely delayed
1402             on-time
5571             on-time
              ...       
3321    severely delayed
3007             on-time
951     severely delayed
1295             delayed
5833             delayed
Name: Status, Length: 5101, dtype: object

3379    severely delayed
3047             on-time
5934             on-time
6103             on-time
1837             delayed
              ...       
729     severely delayed
3845    severely delayed
6642             delayed
6720             delayed
5479             on-time
Name: Status, Length: 1701, dtype: object

In [44]:
model = LogisticRegression(fit_intercept = True, solver='newton-cg', multi_class = 'ovr', penalty = 'l2')
# model = LogisticRegression(fit_intercept = True, solver='lbfgs', multi_class = 'ovr', penalty = 'none', max_iter = 1000)
# model = LogisticRegression(fit_intercept = True, solver='lbfgs', multi_class = 'multinomial', penalty = 'none', max_iter = 1000)
# model = LogisticRegression(fit_intercept = True, solver='newton-cg', multi_class = 'multinomial', penalty = 'none', max_iter = 10000)

# While using multiclass case do multi_class = 'ovr' or 'auto'; can also try other solvers
# While doing regularization, use penalty = 'l2' and also C = 10.0 (need to try other values too)

model.fit(X_train, y_train) 

# The following gives the mean accuracy on the given data and labels
model.score(X_train, y_train) 

# This is the coefficient Beta_1, ..., Beta_7
model.coef_

# This is the coefficient Beta_0
model.intercept_

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:416: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:457: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


LogisticRegression(multi_class='ovr', solver='newton-cg')

0.5712605371495785

array([[-1.16376429e-18, -2.13122620e-33, -1.24782008e-35,
        -1.65909327e-33, -2.12952478e-37, -5.74228719e-38,
        -2.44023732e-35, -2.10178041e-35, -3.88337238e-35,
        -1.75415072e-35, -4.64569447e-35, -1.46438513e-35,
        -4.53449834e-35,  7.14562186e-38,  3.33479662e-38,
        -6.24764009e-37, -1.13310745e-36],
       [-8.47248689e-18, -1.42259033e-32, -7.66827889e-35,
        -1.80958745e-32, -1.78583268e-36, -2.40529819e-37,
        -2.17498710e-34, -2.72944724e-34, -4.49782631e-34,
        -2.70318251e-34, -5.29510687e-34, -2.24872214e-34,
        -6.71900104e-34, -5.33388257e-38, -2.93642275e-38,
        -3.36545877e-36, -1.47734605e-35],
       [ 1.16627715e-19,  1.11863192e-04,  9.39931519e-07,
         3.64891175e-05, -9.59383752e-09,  2.68938170e-10,
        -2.32713185e-06, -3.92125256e-07,  3.20091610e-07,
        -1.49046588e-06,  2.85066221e-07, -1.44439095e-06,
        -9.12643925e-08, -6.12145142e-09, -1.06911624e-08,
         5.53678504e-08, -3.6

array([-1.65306727e-36, -1.82216223e-35,  3.48635913e-08, -3.56889717e-36])

In [158]:
# Predict on new data
custom_parser1 = lambda y: datetime.strptime(y, "%m/%d/%Y")
new_data = pd.read_csv('new1.csv')
final_data = pd.read_csv('Initial.csv')
new_data['Date'] = pd.to_datetime(new_data['Date'])
new_data['Flight Number'] = new_data['Flight Number'].str.replace('UA ', '')
new_data = new_data.rename(columns={'windspeed_80m (km/h)':'windspeed_100m (km/h)','Status (Early, On-time, Late, Severly Late)':'Status'})


new_data = new_data.drop(columns=['Day','Unnamed: 0','Unnamed: 0.1','Scheduled departure hour'])
new_data['Date'] = pd.to_numeric(pd.to_datetime(new_data['Date']))
new_data['Arrival Time'] = pd.to_datetime(new_data['Arrival Time']).dt.strftime('%H:%M')

In [159]:
def str_to_time(x):
    return datetime.strptime(x, '%H:%M').time()

new_data['Arrival Time'] = new_data['Arrival Time'].astype(str).apply(lambda x: datetime.strptime(x, "%H:%M").time())

new_data['Scheduled Arrival(Hrs)'] = new_data['Arrival Time'].apply(lambda x: x.hour)
new_data['Scheduled Arrival(Mins)'] = new_data['Arrival Time'].apply(lambda x: x.minute)
new_data = new_data.drop(columns=['Arrival Time','Origin AirportArr'])
new_data['Status'] = new_data['Status'].fillna(' ')

In [160]:
from sklearn.preprocessing import OneHotEncoder

def get_ohe(df, col):
    ohe = OneHotEncoder(drop=None, handle_unknown='error', sparse=False, dtype='int')
    ohe.fit(df[[col]])
    temp_df = pd.DataFrame(data=ohe.transform(df[[col]]), columns=ohe.get_feature_names_out())
    # If you have a newer version, replace with columns=ohe.get_feature_names_out()
    df.drop(columns=[col], axis=1, inplace=True)
    df = pd.concat([df.reset_index(drop=True), temp_df], axis=1)
    return df

#flight_data = get_ohe(flight_data, 'Carrier Code_Arr')
new_data = get_ohe(new_data, 'Origin Airport')
new_data['Scheduled Departure'] = pd.to_datetime(new_data['Scheduled departure time'])
new_data['Scheduled Departure'] = new_data['Scheduled Departure'].dt.strftime('%H:%M')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [161]:
def str_to_time(x):
    return datetime.strptime(x, '%H:%M').time()

new_data['Scheduled Departure'] = new_data['Scheduled Departure'].astype(str).apply(lambda x: datetime.strptime(x, "%H:%M").time())

new_data['Scheduled Departure(Hrs)'] = new_data['Scheduled Departure'].apply(lambda x: x.hour)
new_data['Scheduled Departure(Mins)'] = new_data['Scheduled Departure'].apply(lambda x: x.minute)
new_data = new_data.drop(columns=['Scheduled Departure','Scheduled departure time','time','Status'])

In [162]:
order = ['Date','Flight Number','temperature_2m (°C)','surface_pressure (hPa)','rain (mm)','snowfall (cm)','weathercode (wmo code)','windspeed_10m (km/h)','windspeed_100m (km/h)','Scheduled Arrival(Hrs)','Scheduled Arrival(Mins)','Scheduled Departure(Hrs)','Scheduled Departure(Mins)','Origin Airport_DEN','Origin Airport_EWR','Origin Airport_IAD','Origin Airport_ORD']

new_data = new_data.reindex(columns=order)
new_data.head(5)
flight_data.head(5)

,Date,Flight Number,temperature_2m (°C),surface_pressure (hPa),rain (mm),snowfall (cm),weathercode (wmo code),windspeed_10m (km/h),windspeed_100m (km/h),Scheduled Arrival(Hrs),Scheduled Arrival(Mins),Scheduled Departure(Hrs),Scheduled Departure(Mins),Origin Airport_DEN,Origin Airport_EWR,Origin Airport_IAD,Origin Airport_ORD
0,1681257600000000000,3839,16.60,992.60,0.00,0,3,14.30,34.40,10,0,7,10,0,0,0,1
1,1681257600000000000,3524,15.40,991.80,0.00,0,0,20.10,37.00,16,52,13,57,0,0,0,1
2,1681257600000000000,538,23.10,989.50,0.00,0,1,26.60,44.30,21,34,18,40,0,0,0,1
3,1681344000000000000,3839,16.20,988.90,0.00,0,0,16.30,31.30,10,0,7,10,0,0,0,1
4,1681344000000000000,3524,15.40,990.00,0.00,0,0,14.50,26.70,16,50,13,55,0,0,0,1


,Date,Flight Number,temperature_2m (°C),surface_pressure (hPa),rain (mm),snowfall (cm),weathercode (wmo code),windspeed_10m (km/h),windspeed_100m (km/h),Scheduled Arrival(Hrs),Scheduled Arrival(Mins),Scheduled Departure(Hrs),Scheduled Departure(Mins),Origin Airport_DEN,Origin Airport_EWR,Origin Airport_IAD,Origin Airport_ORD,Status
0,567993600000000000,554,-16.60,"1,007.20",0.00,0.00,1,17.30,24.90,19,33,16,56,0,0,0,1,on-time
1,567993600000000000,700,-14.90,"1,007.20",0.00,0.00,1,19.90,27.90,22,34,19,57,0,0,0,1,delayed
2,568080000000000000,554,-11.90,"1,005.70",0.00,0.00,0,18.50,31.80,19,33,16,56,0,0,0,1,on-time
3,568080000000000000,656,-16.90,"1,006.30",0.00,0.00,0,14.10,29.50,16,16,13,44,0,0,0,1,on-time
4,568080000000000000,700,-6.30,"1,002.80",0.00,0.00,0,23.10,34.50,22,34,19,57,0,0,0,1,on-time


In [163]:
test_output = pd.DataFrame(model.predict(new_data),columns=["Status"])
test_output.head(10)
print('Percentage of correct predictions is ')
print(model.score(X_test, y_test))
test_output.shape[0]

,Status
0,on-time
1,on-time
2,on-time
3,on-time
4,on-time
5,on-time
6,on-time
7,on-time
8,on-time
9,on-time


Percentage of correct predictions is 
0.5755437977660199


30

In [164]:
new_rows = pd.DataFrame({'Status': [' ', 'on-time']})
test_output = test_output.append(new_rows, ignore_index=True)
test_output.head(32)

/var/folders/c5/91lvrzdx1bzfzwqs8t28758r0000gn/T/ipykernel_2703/2402797443.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_output = test_output.append(new_rows, ignore_index=True)


,Status
0,on-time
1,on-time
2,on-time
3,on-time
4,on-time
5,on-time
6,on-time
7,on-time
8,on-time
9,on-time


In [165]:
merged_df = pd.concat([new_data, test_output], axis=1)
merged_df['Date'] = pd.to_datetime(merged_df['Date'], unit='ns')

# convert datetime to string in original format
merged_df['Date'] = merged_df['Date'].dt.strftime('%m/%d/%Y')
merged_df.head(5)

,Date,Flight Number,temperature_2m (°C),surface_pressure (hPa),rain (mm),snowfall (cm),weathercode (wmo code),windspeed_10m (km/h),windspeed_100m (km/h),Scheduled Arrival(Hrs),Scheduled Arrival(Mins),Scheduled Departure(Hrs),Scheduled Departure(Mins),Origin Airport_DEN,Origin Airport_EWR,Origin Airport_IAD,Origin Airport_ORD,Status
0,04/12/2023,3839,16.60,992.60,0.00,0.00,3.00,14.30,34.40,10.00,0.00,7.00,10.00,0.00,0.00,0.00,1.00,on-time
1,04/12/2023,3524,15.40,991.80,0.00,0.00,0.00,20.10,37.00,16.00,52.00,13.00,57.00,0.00,0.00,0.00,1.00,on-time
2,04/12/2023,538,23.10,989.50,0.00,0.00,1.00,26.60,44.30,21.00,34.00,18.00,40.00,0.00,0.00,0.00,1.00,on-time
3,04/13/2023,3839,16.20,988.90,0.00,0.00,0.00,16.30,31.30,10.00,0.00,7.00,10.00,0.00,0.00,0.00,1.00,on-time
4,04/13/2023,3524,15.40,990.00,0.00,0.00,0.00,14.50,26.70,16.00,50.00,13.00,55.00,0.00,0.00,0.00,1.00,on-time


In [166]:
#Submission

final_data = pd.concat([final_data, test_output], axis=1)
final_data = final_data.drop(columns=['Status (Early, On-time, Late, Severly Late)'])
final_data.head()

,Date,Day,Origin Airport,Flight Number,Arrival Time,Status
0,4/12/2023,Wednesday,ORD,UA 3839,10:00 AM,on-time
1,4/12/2023,Wednesday,ORD,UA 3524,4:52 PM,on-time
2,4/12/2023,Wednesday,ORD,UA 538,9:34 PM,on-time
3,4/13/2023,Thursday,ORD,UA 3839,10:00 AM,on-time
4,4/13/2023,Thursday,ORD,UA 3524,4:50 PM,on-time


In [168]:
final_data.to_csv('April (12-15) predictions.csv')
merged_df.to_csv('(Detailed) April (12-15) predictions.csv')